# Visualize SWOT Sword of Science River Discharge data products with Folium [DRAFT]

In [1]:
import datetime

import branca.colormap as cm
import folium
import geopandas as gpd
import netCDF4
import numpy as np
import pandas as pd
import shapely

## Visualizing the data

You can visualize the mean discharge for a specific Flow Law Paramter Estimation (FLPE) algorithm for a specific river name.

# Table of Modules (Algorithms) and Discharge variables

The following lists the algorithms alongside their discharge variables and location in the SoS assuming that the SoS is an open file represented by the `results` variable.

| Module (Algorithm)               | Discharge Variable               | Location in the SoS              |
|----------------------------------|----------------------------------|----------------------------------|
| HiVDI                            | Q                                | results["hivdi"]["Q"]            |
| MetroMan                         | allq                             | results["metroman"]["allq"]      |
| MOMMA                            | Q                                | results["momma"]["Q"]            |
| neoBAM                           | q1, q2, or q3                    | results["neobam"]["q"]["q1"]     |
| SAD                              | Qa                               | results["sad"]["Qa"]             |
| SIC4DVar                         | Q_mm                             | results["sic4dvar"]["Q_mm"]      |
| MOI HiVDI                        | q                                | results["moi"]["hivdi"]["q"]     |
| MOI MetroMan                     | q                                | results["moi"]["metroman"]["q"]  |
| MOI MOMMA                        | q                                | results["moi"]["momma"]["q"]     |
| MOI neoBAM                       | q                                | results["moi"]["qeobam"]["q"]    |
| MOI SAD                          | q                                | results["moi"]["sad"]["q"]       |
| MOI SIC4DVar                     | q                                | results["moi"]["sic4dvar"]["q"]  |

In [2]:
# Constants

# Select a river
RIVER_NAME = "Ohio River"

# Select a discharge algorithm (hivdi, neobam, metroman, momma, sad, sic4dvar)
DISCHARGE_ALGORITHM = "hivdi"
DISCHARGE_VARIABLE = "Q"

In [3]:
# Open the results file which contains discharge values
results = netCDF4.Dataset("/path/to/na_sword_v16_SOS_results.nc", format="NETCDF4")

In [4]:
# Get discharge for a specific river name
river_names = results['reaches']['river_name'][:]
reach_idx = np.where(river_names[:] == RIVER_NAME)

# Filter out missing values
discharge = results[DISCHARGE_ALGORITHM][DISCHARGE_VARIABLE][:][reach_idx]
missing = results[DISCHARGE_ALGORITHM][DISCHARGE_VARIABLE].missing_value

# Loop through each reach and filter out places where the missing value is present
data_indexes = []
i = 0
for d in discharge:
    is_missing = np.all(d == missing)
    if not is_missing:
        data_indexes.append(i)
    i += 1
print(f"Indexes for locations that have values:\n {data_indexes}")

discharge = discharge[data_indexes]    
# print(f"Discharge values:\n {discharge}")

Indexes for locations that have values:
 [0, 2, 3, 4, 6, 7, 8, 10, 11, 12, 15, 17, 18, 20, 23, 32, 33, 44, 47, 65, 66, 72, 78, 83, 90, 97]


In [5]:
# Get the coordinates of the specific river name reach identifiers
x = results["reaches"]["x"][:][reach_idx][data_indexes]
y = results["reaches"]["y"][:][reach_idx][data_indexes]

In [6]:
results.close()

In [7]:
# Take the mean of the algorithm's river discharge - requires a loop because of ragged arrays
mean_discharge = []
for d in discharge:
    d[d == missing] = np.nan
    mean_discharge.append(np.nanmean(d))    # Ignore NaNs
print(f"Mean discharge:\n {mean_discharge}")

Mean discharge:
 [6292.368494436216, 11629.821333492902, 6016.140519472628, 5934.835287925041, 5968.6940316042055, 9445.687491381777, 2613.4732544782487, 2756.749493185854, 3544.183239303054, 3901.172229662184, 4241.988613309108, 4384.42027456864, 4270.337007732047, 3081.7260264738593, 15561.922897228076, 2450.406759995062, 2811.918509188233, 2695.3232703057643, 2756.7467642950855, 2182.079015920621, 2102.71890444225, 1154.8464735212647, 1955.7366342194973, 2782.5571168647816, 2216.666105239617, 1224.791482748544]


In [15]:
# Visualize mean discharge
max_x = np.max(x)
max_y = np.max(y)
m = folium.Map([max_y, max_x], zoom_start=6, tiles="cartodb positron")

# Color map
min_d = np.min(mean_discharge)
max_d = np.max(mean_discharge)
linear = cm.LinearColormap(["green", "yellow", "red"], vmin=min_d, vmax=max_d)

# # GeoJSON
# folium.GeoJson(geo_df).add_to(m)

# Color line
color_line = folium.ColorLine(
    positions=list(zip(y, x)),
    colors=mean_discharge,
    colormap=linear,
    weight=3,
).add_to(m)

In [16]:
m